In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install python-docx Faker Mimesis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 42.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 56.4 MB/s eta 0:00:00:00:0100:01


In [ ]:
import os
import random
from PIL import Image
from docx import Document
from docx.shared import Pt, Cm, Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.table import WD_ALIGN_VERTICAL
from docx.enum.section import WD_ORIENT
import matplotlib.pyplot as plt
from faker import Faker
from mimesis import Generic
from docx.oxml import parse_xml
from docx.oxml.ns import nsdecls

faker = Faker('ru_RU')
generic = Generic('ru')

os.makedirs('docx', exist_ok=True)
os.makedirs('images', exist_ok=True)

def set_table_style(table, border_option, color_mode, text_size, paragraph_alignment, cell_alignment, font_name):
    vertical_alignment_choices = [
        WD_ALIGN_VERTICAL.TOP,
        WD_ALIGN_VERTICAL.CENTER,
        WD_ALIGN_VERTICAL.BOTTOM
    ]

    for row in table.rows:
        for cell in row.cells:
            cell_paragraph = cell.paragraphs[0]
            cell_paragraph.alignment = cell_alignment
            cell.vertical_alignment = random.choice(vertical_alignment_choices)

            if color_mode == 'color':
                color_value = "%06x" % random.randint(0, 0xFFFFFF)
                cell._element.get_or_add_tcPr().append(
                    parse_xml(
                        f'<w:shd {nsdecls("w")} w:fill="{color_value}"/>'
                    )
                )

            data_type = random.choice(['date', 'number', 'word'])

            if data_type == 'date':
                text = str(generic.datetime.date())
            elif data_type == 'number':
                text = str(random.randint(1, 100))
            elif data_type == 'word':
                text = faker.word()

            if not cell_paragraph.runs:
                run = cell_paragraph.add_run(text)
            else:
                run = cell_paragraph.runs[0]
                run.text = text
            run.font.size = Pt(text_size)
            run.font.name = font_name

    tbl = table._element

    if border_option == "full":
        for tcPr in tbl.xpath(".//w:tcPr"):
            tcBorders = parse_xml(r"""
            <w:tcBorders xmlns:w="http://schemas.openxmlformats.org/wordprocessingml/2006/main">
                <w:left w:val="single" w:sz="4" w:space="0" w:color="auto"/>
                <w:right w:val="single" w:sz="4" w:space="0" w:color="auto"/>
                <w:top w:val="single" w:sz="4" w:space="0" w:color="auto"/>
                <w:bottom w:val="single" w:sz="4" w:space="0" w:color="auto"/>
            </w:tcBorders>
            """)
            tcPr.append(tcBorders)
    elif border_option == "none":
        for tc in tbl.xpath(".//w:tcBorders"):
            tc.getparent().remove(tc)
    elif border_option == "sides":
        for tcPr in tbl.xpath(".//w:tcPr"):
            tcBorders = parse_xml(r"""
            <w:tcBorders xmlns:w="http://schemas.openxmlformats.org/wordprocessingml/2006/main">
                <w:left w:val="single" w:sz="4" w:space="0" w:color="auto"/>
                <w:right w:val="single" w:sz="4" w:space="0" w:color="auto"/>
                <w:top w:val="single" w:sz="4" w:space="0" w:color="auto"/>
                <w:bottom w:val="single" w:color="auto"/>
            </w:tcBorders>
            """)
            tcPr.append(tcBorders)

        for tcBorders in tbl.xpath(".//w:tcBorders"):
            for border in tcBorders:
                border_type = border.tag.split('}')[-1]
                if border_type == "left" or border_type == "right":
                    tcBorders.remove(border)

    table.alignment = paragraph_alignment

def generate_color_square_image(filepath):
    colors = [
        (255, 0, 0), (0, 255, 0), (0, 0, 255),
        (255, 255, 0), (128, 0, 128), (255, 165, 0),
        (255, 192, 203), (0, 255, 255), (255, 0, 255)
    ]
    
    chosen_color = random.choice(colors)
    image = Image.new("RGB", (200, 200), chosen_color)
    image.save(filepath)

def generate_chart(filepath, chart_type='pie'):
    if chart_type == 'pie':
        labels = ['A', 'B', 'C', 'D']
        sizes = [15, 30, 45, 10]
        colors = ['gold', 'yellowgreen', 'lightcoral', 'lightskyblue']
        plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140)
        plt.axis('equal')
    elif chart_type == 'bar':
        categories = ['Cat1', 'Cat2', 'Cat3']
        values = [5, 20, 15]
        plt.bar(categories, values, color=['red', 'blue', 'green'])
    elif chart_type == 'line':
        x = [0, 1, 2, 3, 4]
        y = [0, 1, 4, 9, 16]
        plt.plot(x, y, marker='o')

    plt.savefig(filepath)
    plt.close()

def add_picture_and_caption(doc, image_path, caption_format, caption_number, caption_font_size, caption_alignment, caption_position):
    random_words = faker.sentence(nb_words=random.randint(2, 5)).rstrip('.')
    caption_text = f"{caption_format.format(caption_number)} {random_words}"

    if caption_position == 'before':
        paragraph = doc.add_paragraph(caption_text)
        paragraph.alignment = caption_alignment
        paragraph.runs[0].font.size = Pt(caption_font_size)

    paragraph = doc.add_paragraph()
    run = paragraph.add_run()
    run.add_picture(image_path, width=Inches(2))
    paragraph.alignment = caption_alignment

    if caption_position == 'after':
        paragraph = doc.add_paragraph(caption_text)
        paragraph.alignment = caption_alignment
        paragraph.runs[0].font.size = Pt(caption_font_size)


def add_list(doc, list_length, list_indent, line_spacing, base_font_size, list_alignment):
    list_style = random.choice([None, 'bold', 'italic', 'bold_italic'])
    use_numbered_list = random.choice([True, False]) 

    for i in range(1, list_length + 1):
        if use_numbered_list:
            list_item_text = f"{i}. {faker.sentence(nb_words=random.randint(3, 6))}"
        else:
            list_item_text = f"• {faker.sentence(nb_words=random.randint(3, 6))}"

        list_item = doc.add_paragraph(list_item_text)
        list_item.paragraph_format.left_indent = Cm(list_indent)
        list_item.paragraph_format.line_spacing = line_spacing

        list_item.runs[0].font.size = Pt(base_font_size)

        if list_style == 'bold':
            list_item.runs[0].bold = True
        elif list_style == 'italic':
            list_item.runs[0].italic = True
        elif list_style == 'bold_italic':
            list_item.runs[0].bold = True
            list_item.runs[0].italic = True

        list_item.alignment = list_alignment


def set_header_footer(doc):
    fonts = ['Times New Roman', 'Arial', 'Calibri', 'Verdana', 'Georgia']
    font_name = random.choice(fonts)
    font_size = random.randint(8, 12)
    
    alignment_choices = [WD_ALIGN_PARAGRAPH.LEFT, WD_ALIGN_PARAGRAPH.CENTER, WD_ALIGN_PARAGRAPH.RIGHT]
    alignment = random.choice(alignment_choices)
    
    font_styles = ['normal', 'italic', 'bold', 'bold_italic']
    font_style = random.choice(font_styles)
    
    random_text = faker.sentence() 
    
    section = doc.sections[0]

    colontitul_option = random.choice([0, 1, 2, 3])  # 0 без колонтитулов, 1 только верхний, 2 только нижний, 3 верхний и нижний

    if colontitul_option == 1 or colontitul_option == 3: 
        header = section.header
        paragraph_header = header.paragraphs[0]
        paragraph_header.text = random_text
        paragraph_header.alignment = alignment
        run = paragraph_header.runs[0]
        run.font.size = Pt(font_size)
        run.font.name = font_name
        
        if font_style == 'italic':
            run.italic = True
        elif font_style == 'bold':
            run.bold = True
        elif font_style == 'bold_italic':
            run.bold = True
            run.italic = True

    if colontitul_option == 2 or colontitul_option == 3:  # вставляем нижний колонтитул
        footer = section.footer
        paragraph_footer = footer.paragraphs[0]
        paragraph_footer.text = random_text 
        paragraph_footer.alignment = alignment
        run = paragraph_footer.runs[0]
        run.font.size = Pt(font_size)
        run.font.name = font_name
        
        if font_style == 'italic':
            run.italic = True
        elif font_style == 'bold':
            run.bold = True
        elif font_style == 'bold_italic':
            run.bold = True
            run.italic = True


    
def create_random_docx(doc_number):
    doc = Document()
    
    set_header_footer(doc)

    section = doc.sections[0]
    if random.choice([True, False]):
        section.orientation = WD_ORIENT.LANDSCAPE
        section.page_width, section.page_height = section.page_height, section.page_width

    base_font_size = random.randint(8, 16)
    title_font_size = base_font_size + random.choice([2, 3, 4])
    
    caption_font_sizes = [base_font_size, base_font_size - 1, base_font_size - 2]
    caption_font_size = random.choice(caption_font_sizes)
    
    fonts = ['Times New Roman', 'Arial', 'Calibri', 'Verdana', 'Georgia']
    font_name = random.choice(fonts)
    
    list_alignment = random.choice([WD_ALIGN_PARAGRAPH.LEFT, WD_ALIGN_PARAGRAPH.CENTER, WD_ALIGN_PARAGRAPH.RIGHT])

    use_numbers = random.choice([True, False])

    use_indent = random.choice([True, False])
    text_alignment = random.choice([
        WD_ALIGN_PARAGRAPH.CENTER,
        WD_ALIGN_PARAGRAPH.LEFT,
        WD_ALIGN_PARAGRAPH.RIGHT,
        WD_ALIGN_PARAGRAPH.JUSTIFY
    ])

    alignment_choices = [
        WD_ALIGN_PARAGRAPH.CENTER,
        WD_ALIGN_PARAGRAPH.LEFT,
        WD_ALIGN_PARAGRAPH.RIGHT,
        WD_ALIGN_PARAGRAPH.JUSTIFY
    ]

    alignment_weights = [0.5, 0.166, 0.167, 0.167]

    align_choice = random.choices(alignment_choices, weights=alignment_weights, k=1)[0]

    table_alignment = random.choice(alignment_choices)
    cell_alignment = random.choice(alignment_choices)

    num_titles = 5

    title_bold = random.choice([True, False])
    title_italic = random.choice([True, False])

    caption_alignment = random.choices(
        [WD_ALIGN_PARAGRAPH.CENTER, WD_ALIGN_PARAGRAPH.LEFT, WD_ALIGN_PARAGRAPH.RIGHT],
        weights=[0.7, 0.15, 0.15],
        k=1
    )[0]

    caption_position = random.choices(['before', 'after'], weights=[0.3, 0.7])[0]

    caption_format = random.choice([
        "Рис. {} ",
        "Рисунок {} - ",
        "Рисунок {} "
    ])

    consistent_table_caption_format = random.choice([
        "Табл. {} ",
        "Таблица {} - ",
        "Таблица. "
    ])

    consistent_caption_style = random.choice(['normal', 'italic', 'bold', 'bold_italic'])

    consistent_table_caption_alignment = random.choice([
        WD_ALIGN_PARAGRAPH.CENTER, 
        WD_ALIGN_PARAGRAPH.LEFT, 
        WD_ALIGN_PARAGRAPH.RIGHT
    ])

    for i in range(num_titles):
        title_number = f"{i+1}. " if use_numbers else ""
        title_text = faker.sentence().rstrip('.')
        title = f"{title_number}{title_text}"
        paragraph = doc.add_paragraph()
        run = paragraph.add_run(title)
        run.bold = title_bold
        run.italic = title_italic
        run.font.size = Pt(title_font_size)
        run.font.name = font_name
        paragraph.alignment = align_choice

        num_paragraphs = random.randint(2, 4)
        total_chars = random.randint(500, 1000)

        num_images = random.randint(1, 2)
        num_graphs = random.randint(0, 1)
        num_tables = 1 
        elements_to_insert = num_images + num_graphs + num_tables

        insert_positions = random.sample(range(num_paragraphs + elements_to_insert), elements_to_insert)

        images_inserted = 0
        graphs_inserted = 0
        tables_inserted = 0
        list_inserted = False  
        for j in range(num_paragraphs + elements_to_insert):
            if j in insert_positions:
                if images_inserted < num_images:
                    image_path = f'images/color_square_{doc_number}_{i}_{images_inserted}.png'
                    generate_color_square_image(image_path)
                    add_picture_and_caption(doc, image_path, caption_format, f"{i+1}.{images_inserted+1}", 
                                            caption_font_size, caption_alignment, caption_position)
                    images_inserted += 1
                elif graphs_inserted < num_graphs:
                    chart_type = random.choice(['pie', 'bar', 'line'])
                    chart_path = f'images/chart_{chart_type}_{doc_number}_{i}_{graphs_inserted}.png'
                    generate_chart(chart_path, chart_type)
                    add_picture_and_caption(doc, chart_path, caption_format, f"{i+1}.{graphs_inserted+1}", 
                                            caption_font_size, caption_alignment, caption_position)
                    graphs_inserted += 1
                elif tables_inserted < num_tables:
                    caption_number = f"{tables_inserted + 1}" if random.choice([True, False]) else ""
                    caption_text = consistent_table_caption_format.format(caption_number) + faker.sentence(random.randint(3, 7)).rstrip('.')
                    
                    caption_paragraph = doc.add_paragraph(caption_text)
                    caption_paragraph.runs[0].font.size = Pt(caption_font_size)

                    if consistent_caption_style == 'italic':
                        caption_paragraph.runs[0].italic = True
                    elif consistent_caption_style == 'bold':
                        caption_paragraph.runs[0].bold = True
                    elif consistent_caption_style == 'bold_italic':
                        caption_paragraph.runs[0].bold = True
                        caption_paragraph.runs[0].italic = True

                    caption_paragraph.alignment = consistent_table_caption_alignment
                    color_mode = random.choice(['color', 'none'])
                    border_option = random.choice(['full', 'none', 'sides'])
                    table = doc.add_table(rows=3, cols=3)
                    set_table_style(table, border_option, color_mode, base_font_size, table_alignment, cell_alignment, font_name)

                    tables_inserted += 1

            else:
                text_length = (total_chars if j == num_paragraphs + elements_to_insert - 1 
                               else random.randint(100, max(100, total_chars - (100 * (num_paragraphs + elements_to_insert - j - 1)))))

                text = faker.text(max_nb_chars=text_length).replace('\n', ' ')
                paragraph = doc.add_paragraph(text)

                if not list_inserted:  
                    list_length = random.randint(2, 5)
                    list_indent = random.choice([0, 1])
                    line_spacing = random.choice([0.75, 1.15])
                    add_list(doc, list_length, list_indent, line_spacing, base_font_size, list_alignment)
                    list_inserted = True  

                run = paragraph.runs[0]
                run.bold = random.choice([True, False])
                run.italic = random.choice([True, False])
                run.font.size = Pt(base_font_size)
                run.font.name = font_name
                paragraph.alignment = text_alignment

                if use_indent:
                    paragraph.paragraph_format.first_line_indent = Cm(1)

                total_chars -= text_length

    doc.save(f'docx/doc_{doc_number}.docx')

for i in range(10):
    create_random_docx(i)